<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/gp_aprendizaje/libreriaGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Modulos y constantes

import numpy as np
import random
import scipy
import scipy.linalg
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import drive
import pandas as pd

In [68]:
# Pruebas con funciones anonimas en python

class funcionesPrueba():
  def __init__(self):
    '''
    Covarianza puede ser una cadena de texto o una funcion anónima como combinación de metodos de la clase
    covarianza es un diccionario
    '''
    self.Kernels = self.kernels()

  def calcCovM(self,x,y):
    '''
    Dados x e y como ndarrays, calcula la matriz de covarianzas cuyos elementos son evaluados de la forma
    C(xi,yi) = (k(xi,yi)) con rango (nx x ny) donde nx y ny son la longitud de los vectores x  e y respectivamente.
    hiper son los hiperparámetros usados en el modelo
    '''
    # Se crean matrices [[x],[x],...] (longitud de la concatenacion nx) y [[y],[y],...].T (longitud de la concatenacion ny)
    xM = np.full((len(y),len(x)),x)
    yM = (np.full((len(x),len(y)),y)).T
    # Llamamos la funcion Ker que nos debe devolver como salida una matriz cuyos elementos calculados representan k(x,y)
    return self.Ker(xM,yM,*self.Hiper)


  def compCov(self,covarianza,*hiper):
    '''
    Permite definir el kernel a usar por el proceso gaussiano, tanto si es uno de los kernels definidos en la clase como si es una combinacion de los mismos
    El argumento es una funcion anónima en caso de ser una composición, en caso contrario es suficiente con proporcionar el nombre del kernel definido
    hiper son los hiperparámetros que precisan el kernel y vienen dado por un diccionario
    '''
    self.Hiper = hiper
    self.Ker = lambda x_test, x_obs, *hiper: covarianza(x_test,x_obs,*self.Hiper) # Covarianza definida

  class kernels():
    '''
    Se definen kernels de uso común junto a sus hiperparámetros
    '''
    def __init__(self):
      return

    def exponencialCuadrada(self,xb,xa,s,l):
      '''
      Correlacion estacionaria (depende solo de la distancia r = abs(xb-xa)) con decaimiento exponencial cuadratico
      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return np.power(s,2)*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def ruidoBlanco(self,xb,xa,sigma):
      '''
      Correlaciones de ruido blanco o gaussiano puro, solo correlaciona los puntos consigo mismo. 
      Representa incertidumbres intrínsecas en las observaciones y asume una distribución a priori de esos errores gaussiano
      Hiperparáámetros:
      s : amplitud o incertidumbre del ruido generado
      '''
      return np.power(sigma,2) if not (xb-xa).any() else 0. # El metodo any se refiere a que si xb y xa son arrays, la operacion logica presentada es ambigua. 
                                                            # Hay que especificar si todos es verdadero o falso según entre todos los elementos de los arrays o elemento - a - elemento
                                                            # .any() es el metodo que evalua el segundo caso

In [67]:
x_test = np.linspace(1.,5.,4)
x_obs = np.linspace(1.,5.,4)
x_obs = x_obs.T
print(x_test,x_obs)
prueba = funcionesPrueba()
s = 3.
l = 4.
sigma = 0.
prueba.compCov(lambda x,y,s,l,sigma: prueba.Kernels.exponencialCuadrada(x,y,s,l)+prueba.Kernels.ruidoBlanco(x,y,sigma),s,l,sigma) # Aqui especificamos los hiperparametros
print(prueba.calcCovM(x_test,x_obs,s,l,sigma))

[1.         2.33333333 3.66666667 5.        ] [1.         2.33333333 3.66666667 5.        ]
[[9.         8.51363522 7.20663663 5.45877594]
 [8.51363522 9.         8.51363522 7.20663663]
 [7.20663663 8.51363522 9.         8.51363522]
 [5.45877594 7.20663663 8.51363522 9.        ]]


In [46]:
print(x_test,x_obs)
x = 3.66666667
y = 5.
a = 3.*3.*np.exp(-np.power((x-y),2)/(2.*4.*4.))
print(a)

[1.         2.33333333 3.66666667 5.        ] [1.         2.33333333 3.66666667 5.        ]
8.513635222525789


In [92]:
class gaussProcess():
  def __init__(self):
    '''
    Los objetos de esta clase modelan procesos gaussianos caracterizados por su promedio y covarianza
    '''
    #self.Derivadas = prior_Partial_Der
    self.Kernels = self.kernels()

  def calcCovM(self,x,y,*hiper):
    '''
    Dados x e y como ndarrays, calcula la matriz de covarianzas cuyos elementos son evaluados de la forma
    C(xi,yi) = (k(xi,yi)) con rango (nx x ny) donde nx y ny son la longitud de los vectores x  e y respectivamente.
    hiper son los hiperparámetros usados en el modelo
    '''
    # Se crean matrices [[x],[x],...] (longitud de la concatenacion nx) y [[y],[y],...].T (longitud de la concatenacion ny)
    xM = np.full((len(y),len(x)),x)
    yM = (np.full((len(x),len(y)),y)).T
    # Llamamos la funcion Ker que nos debe devolver como salida una matriz cuyos elementos calculados representan k(x,y)
    return self.Ker(xM,yM,*hiper)

  def compCov(self,covarianza,*hiper):
    '''
    Permite definir el kernel a usar por el proceso gaussiano, tanto si es uno de los kernels definidos en la clase como si es una combinacion de los mismos
    El argumento es una funcion anónima en caso de ser una composición, en caso contrario es suficiente con proporcionar el nombre del kernel definido
    hiper son los hiperparámetros que precisan el kernel y vienen dado por un diccionario
    '''
    self.Hiper = hiper
    self.Ker = lambda x_test, x_obs, *hiper: covarianza(x_test,x_obs,*self.Hiper) # Covarianza definida


  class kernels():
    '''
    Se definen kernels de uso común junto a sus hiperparámetros
    '''
    def __init__(self):
      return

    def exponencialCuadrada(self,xb,xa,s,l):
      '''
      Correlacion estacionaria (depende solo de la distancia r = abs(xb-xa)) con decaimiento exponencial cuadratico

      k(xb,xa) = s² exp( - (xb-xa)²/(2 l²) )

      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return np.power(s,2)*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def ruidoBlanco(self,xb,xa,sigma):
      '''
      Correlaciones de ruido blanco o gaussiano puro, solo correlaciona los puntos consigo mismo. 

      k(xb,xa) = sigma² delta(xb,xa)

      Representa incertidumbres intrínsecas en las observaciones y asume una distribución a priori de esos errores gaussiano
      Hiperparáámetros:
      s : amplitud o incertidumbre del ruido generado
      '''
      return np.power(sigma,2) if np.equal(xb,xa) else 0. # El metodo any se refiere a que si xb y xa son arrays, la operacion logica presentada es ambigua. 
                                                          # Hay que especificar si todos es verdadero o falso según entre todos los elementos de los arrays o elemento - a - elemento
                                                          # .any() es el metodo que evalua el segundo caso

  def cholDescomp(self,K):
    '''
    Descomposicion cholesky de una matriz dada
    '''
    try:
      L = scipy.linalg.cholesky(K, lower=True)
    except:
      L = scipy.linalg.cholesky(K + np.diag(0.000001*np.ones(int(np.sqrt(K.size)))), lower=True)
      raise NameError("No es posible efectuar descomposicion cholesky")

    return L

  def calcInvK(self,matriz):
    '''
    Calculo de la inversa de matriz. 
    '''
    # Calculo de matriz inversa de la covarianza. 
    try:
      invK = np.linalg.inv(matriz)
    except:
      raise NameError("No es posible calcular matriz inversa")

    return invK

  def condicionarGP(self,dominio,observaciones,*dominio_test):
    '''
    Dadas unas observaciones junto a sus valores de dominio (uni-dimensionales) en el rango sobre el que se modela el GP, 
    obtiene la covarianza asociada a la distribución de trayectorias condicionadas al conjunto de partida dado un dominio 
    de test en el que queremos comprobar la predicción. Por tanto permite generar las trayectorias compatibles con los datos

    También calcula el logaritmo de la probabilidad de verosimilitud marginal (marginal likelihood) de las observaciones dados los inputs 
    y parámetros del modelo usado.

    Este algoritmo esta especificado en el libro "Gaussian Processes for Machine Learning", como algoritmo 2.1
    '''
    self.xtrain = dominio
    self.ytrain = observaciones
    self.xtest = dominio_test

    # Definimos nueva covarianza
    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priori,
    # para lo cual calculamos la matriz de covarianza asociada al dominio de las observaciones sobre si mismo
    # y al cruce del dominio de las observaciones y del test de prediccion.
    try:
      self.Cov_obs_obs = self.calcCovM(self.xtrain,self.xtrain,*self.Hiper)
      self.Cov_obs_test = self.calcCovM(self.xtrain,self.xtest,*self.Hiper)
    except:
      raise NameError("Debes especificar primero el kernel que estas usando junto a sus hiperparametros con el metodo self.compCov")

    cholL = np.linalg.cholesky(self.Cov_obs_obs)

    # Calculo del vector alfa como solucion del sistema K_obs_obs*alfa = y_entrenamiento usando la descomposición cholesky anterior
    self.Alfa = scipy.linalg.cho_solve((cholL,True),self.ytrain)

    # Vector v solución particular del sistema cholL*v = k_test para todos los datos del entrenamiento.
    # Este sistema se resuelve introduciendo k_star no como matriz columna sino como la submatriz calculada como la evaluación de la covarianza a priori de los inputs de los valores a predecir y los inputs del entrenamiento
    v = scipy.linalg.solve_triangular(cholL,self.cov_obs_test, lower=True)

    # Se obtiene el valor promedio del proceso predictivo condicionado a las observaciones
    self.Media_pred = np.matmul(self.Cov_obs_test.T,self.Alfa)

    # Covarianza predictiva o condicionada. El proceso llevado a cabo ha sido obtener la distribución condicionada del GP a las observaciones
    # Intenta calcular directamente la matriz, en caso contrario calcular la matriz de covarianza de la distribucion a priori dado el test
    try:
      self.Cov_pred = self.Cov_priori - np.matmul(v.T,v)
    except:
      # En caso que no esté definida o no coincida con la dimensionalidad del test, se recalcula la covarianza a priori
      self.Cov_priori = self.calcCovM(self.xtest,self.xtest,*self.Hiper)
      self.Cov_pred = self.Cov_priori - np.matmul(v.T,v)

    # La siguiente matriz descompuesta es necesaria para generar procesos del GP.
    self.L_pred = self.cholDescomp(self.Cov_pred)
  

  def log_prob_verosimilitud_datos(self,dominio,observaciones):
    '''
    Dadas unas observaciones, kernel e hiperparámetros dados, calcula el logaritmo de la probabilidad de verosimilitud de las observaciones al modelo y sus parámetros
    '''

    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priorir y el término de ruido.
    print(self.calcCovM(dominio,dominio,*self.Hiper))
    cholL = np.linalg.cholesky(self.calcCovM(dominio,dominio,*self.Hiper))

    # Calculo del vector alfa como solucion del sistema K*alfa = y_entrenamiento usando la descomposición cholesky anterior
    alfa = scipy.linalg.cho_solve((cholL,True),observaciones)

    # Calculo de la probabilidad de verosimilitud marginal a este modelo y sus parámetros
    self.log_marg_y = -np.sum(np.log(np.diag(cholL)),axis=0)
    self.log_marg_y = self.log_marg_y - 0.5*np.dot(alfa,observaciones)
    self.log_marg_y = self.log_marg_y - (float(len(dominio)/2.))*np.log(2*np.pi)

  def prioriGP(self,x_test):
    '''
    Computa los términos necesarios para extraer realizaciones del GP siguiendo la distribución a priori
    '''
    try:
      self.Cov_priori = self.calcCovM(self.xtest,self.xtest,*self.Hiper)
    except:
      raise NameError("Debes especificar primero el kernel que estas usando junto a sus hiperparametros con el metodo self.compCov")

    self.xtest = x_test

    # La siguiente matriz descompuesta es necesaria para generar procesos del GP.
    self.L_priori = self.cholDescomp(self.Cov_pred)


  def procesosPrioriGP(self):
    '''
    Genera realizaciones del proceso gaussiano dada la descomposicion cholesky de la matriz de covarianzas y la media correspondiente
    '''
    # Si no se ha calculado la descomposicióón de la covarianza se llama al método aqui
    try:
      return self.Mean_priori +  np.matmul(self.L_priori,(np.randn(len(self.xtest))).T)
    except:
      raise NameError("No se ha incluido los valores del dominio sobre los que muestrear el proceso o no se ha calculado la descomposición de la covarianza")

  def procesosCondicionadosGP(self):
    '''
    Genera realizaciones del proceso gaussiano dada la descomposicion cholesky de la matriz de covarianzas y la media correspondiente
    '''
    try:
      return self.Media_pred + np.matmul(self.L_pred,(np.randn(len(self.xtest))).T)
    except:
      raise NameError("El GP no ha sido condicionado a las observaciones")

  def sigmaCalc(self,cov):
    '''
    Desviacion estándar punto a punto del proceso gausiano dado por la matriz de covarianza 
    Es la raiz cuadrado de los elementos de la diagonal de la matriz de covarianzas del proceso generado
    '''
    return np.sqrt(np.diag(cov))


  def correlacionPuntoPunto(self):
    '''
    Obtiene la correlacion punto a punto dado un inputo concreto por el indice i_x
    '''
    try:
      self.corrPP_pred = np.array([ self.K_pred[i,:] for i in range(len(self.K_pred)) ])
    except:
      print(" Proceso gaussiano no entrenado aún, no existe K_pred ")
    self.corrPP_prior = np.array([ self.K_prior[i,:] for i in range(len(self.K_prior)) ])

  def derivadaVerosimilitudMarginal(self,*args):
    '''
    Dadas unas observaciones particulares estima la derivada del logaritmo de la probabilidad asociada a la distribución de verosimilitud marginal asociada al modelo respecto sus hiperparámetros
    '''

    if not isinstance(args[0], np.ndarray):
      raise NameError("Las observaciones dadas para calcular la derivada del logaritmo de la distribución marginal de verosimilitud no es un array con datos")
    else:
      observaciones = args[0]
      observaciones = observaciones.reshape(len(observaciones),1) # Esta operacion es necesaria para que mas tarde el producto matmul nos reproduzca una matriz en lugar de producto escalar entre vectores.

    # Vector auxiliar alpha
    try:
      alpha = np.matmul(self.invK,observaciones)
    except:
      raise NameError("Necesitas calcular la matriz inversa de covarianzas de la distribución a priori")

    alpha_alpha = np.matmul(alpha,alpha.T) 
    alpha_alpha_minus_invK = alpha_alpha - self.invK

    # Calculo. Es el valor de la derivada del logaritmo
    var_incSignal = self.Ruido*np.sum(alpha_alpha_minus_invK)# Si asumimos como hiperparámetro la amplitud del ruido sobre las medidas respecto la señal verdadera, se debe añadir este término
    self.derVerMarg = [ 0.5*np.trace(np.matmul(alpha_alpha_minus_invK,self.Derivadas[i])) for i in range(len(self.Derivadas))]
    self.derVerMarg.append(var_incSignal)
    self.derVerMarg = np.asarray(self.derVerMarg)
  
  def derivada_LO_CV(self,*args):
    '''
    Dadas unas observaciones particulares estima la derivada de la suma de los logaritmos de la evaluación de la distribución de probabilidad marginal de verosimilitud a que subconjuntos
    de las observaciones están condicionadas al resto, respecto a los hiperparámetros del modelo. Si se encuentra el máximo de este valor, estamos asegurándonos de que con los hiperparámetros ajustados
    las "predicciones" tras generar trayectorias del proceso gausiano que corresponderían a los subconjuntos excluidos sucesivamente son las más probables que el modelo puede hacer. Al ser la suma, esta optimización 
    es global y abarca la aproximación de observacion/test de todos los subconjuntos.
    '''

    if not isinstance(args[0], np.ndarray):
      raise NameError("Las observaciones dadas para calcular la derivada del logaritmo de la distribución marginal de verosimilitud no es un array con datos")
    else:
      observaciones = args[0]

    # Vector auxiliar alpha y matriz auxiliar Z y la diagonal de la inversa de la distribucion a priori
    try:
      alpha = np.matmul(self.invK,observaciones)
      invK_diag = np.diag(self.invK)
      zeta = [np.matmul(self.invK,self.Derivadas[i]) for i in range(len(self.Derivadas)) ] # Es una lista de matrices
      zeta_K = [np.diag(np.matmul(zeta[i],self.invK)) for i in range(len(self.Derivadas)) ]
      alpha_invK = np.multiply(alpha,1./invK_diag)
    except:
      raise NameError("Necesitas calcular la matriz inversa de covarianzas de la distribución a priori")
    
    termino_1 = [np.matmul(np.matmul(alpha_invK,zeta[i]),alpha) for i in range(len(self.Derivadas)) ] # Debido a Z esto es una lista
    termino_2 = [np.sum(np.multiply(np.multiply(0.5*(1. + np.multiply(np.power(alpha,2),1./invK_diag)),zeta_K[i]),1./invK_diag)) for i in range(len(self.Derivadas)) ] # Debido a zeta_K esto es una lista

    # Calculo. Es el valor de la derivada del logaritmo
    self.derVerLOO = np.asarray([ termino_1[i] + termino_2[i] for i in range(len(self.Derivadas))])
    

In [85]:


class datosVisualizacion():
  '''
  Esta clase contiene los datos de entrenamiento y realizaciones de los procesos gaussianos para la visualización de los mismos
  '''
  def __init__(self,Titulo):
    self.x = []
    self.y = []
    self.sigma = []
    self.realizacion_dict = {}
    self.titulo = Titulo
    self.inicializacionPlot()

  def inicializacionPlot(self):
    '''
    Inicializa el objeto matplotlib
    '''
    self.fig,self.ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    self.ax.set_title(self.titulo)
    self.fig_hm, self.ax_hm = plt.subplots(nrows=1,ncols=1,figsize=(10,10))

  def add_covariance(self, cov):
    '''
    Añada matriz de covarianza y alternativamente calcula la covarianza del proceso gaussiano sobre el dominio
    '''

    self.cov_GP = cov

  def add_mean(self,X,Mean):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_mean = X
    self.mean = Mean


  def add_sigma(self,X,Sigma):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_sigma = X
    self.sigma = Sigma
    self.meanMarker = 'r-'

  def add_train_data(self,X,Y):
    self.xtrain = X
    self.ytrain = Y
    self.marker = 'b*'

  def add_data(self,X,Y,label):
    self.x.append(X)
    self.y.append(Y)
    self.realizacion_dict[label] = len(self.x)-1

  def add_corrPuntoPunto(self,corrPP, corrInputs):
    '''
    Añade correlaciones punto a punto sobre un dominio dado por los valores corrInputs
    '''
    self.corrPP = corrPP
    self.corr_in = corrInputs

  def add_data_plot(self,label):
    try:
      self.ax.plot(self.x[self.realizacion_dict[label]],self.y[self.realizacion_dict[label]],'g--')
    except:
      print('No existe datos referentes a etiqueta :',label)

  def add_train_data_plot(self):
    try:
      self.ax.plot(self.xtrain,self.ytrain,self.marker)
    except:
      print('No existen datos de entrenamiento')

  def add_mean_plot(self):
    try:
      self.ax.plot(self.x_mean,self.mean,self.meanMarker)
    except:
      print('No existen datos de entrenamiento')



  def contorno_sigma(self):
    '''
    Muestra en pantalla contorno de 1 y 2 veces los valores de sigma del proceso gausiano
    '''
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.9600 * self.sigma,
                             (self.mean + 1.9600 * self.sigma)[::-1])),
             alpha=.45, fc='y', ec='None', label='95% confidence interval')
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.000 * self.sigma,
                             (self.mean + 1.000 * self.sigma)[::-1])),
             alpha=.35, fc='b', ec='None', label='68% confidence interval')
    
  def plotCorrPuntoPunto(self,indices):
    '''
    Visualiza correlaciones punto a punto para los inputs dados por sus indices en la lista indices como argumento del metodo
    '''

    ' Generamos y visualizamos las imagen directamente en este metodo'
    self.fig_cpp, self.ax_cpp = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    try:
      [self.ax_cpp.plot(self.corr_in,self.corrPP[indice],'-') for indice in indices] 
    except:
      print('No existe datos referentes referentes a correlaciones punto a punto')
    self.ax_cpp.grid(True)
    self.fig_cpp.show()



  def mostrarPlot(self):
    '''
    Muestra plot
    '''
    self.add_mean_plot()
    self.add_train_data_plot()
    self.contorno_sigma()
    self.ax.grid(True)
    self.fig.show()

  def hmapCov(self):
    '''
    Visualiza covarianza como campo bidimensional
    '''
    plotRange = np.arange(len(self.cov_GP))
    heatmap = self.ax_hm.pcolormesh(self.cov_GP,cmap='RdBu_r')
    self.ax_hm.axis([plotRange.min(), plotRange.max(), plotRange.max(), plotRange.min()])
    self.fig_hm.colorbar(heatmap, ax=self.ax_hm)
    self.fig_hm.show()




In [93]:
x = np.linspace(0,10,200)

x_train = np.linspace(0.1,9.7,40)
y_train = 1.1*np.sin(2.*x_train) + np.asarray([random.gauss(0.,1.) for i in range(len(x_train))])


grp = 5
ss = np.linspace(0.1,2.,grp)
ls = np.linspace(0.1,3.,grp)

valores_s = np.zeros((grp,grp))
valores_l = np.zeros((grp,grp))
probabilidades = np.zeros((grp,grp))

sigma = 1.

gp_prueba = gaussProcess()


i = 0
for s in ss:

  j = 0
  for l in ls:
    # Generamos un proceso gausiano para generar la distribucion de funciones con probabilidades a priori (las que nos ofrece las incertidumbres en los parametros)
    gp_prueba.compCov(lambda x,y,s,l,sigma: gp_prueba.Kernels.exponencialCuadrada(x,y,s,l)+gp_prueba.Kernels.ruidoBlanco(x,y,sigma),s,l,sigma) # Aqui especificamos los hiperparametros


    gp_prueba.log_prob_verosimilitud_datos(x_train,y_train)
    print("s :", s)
    print("l :", l)
    print("logaritmo de la probabilidad de verosimilitud :",gp_prueba.log_marg_y)
    probabilidades[i,j] = gp_prueba.log_marg_y
  
    j = j + 1
  
  i = i + 1

fig_3, ax_3 = plt.subplots(1,1)



ax_3.imshow(probabilidades)

fig_3.show()

ValueError: ignored